# Temporal and Spatial Analysis of Red Alerts in Israel

In [128]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
from collections import Counter
from scipy.interpolate import make_interp_spline
import numpy as np

# Data extraction

The data used in this analysis was obtained from the [Cumta Telegram channel](https://t.me/CumtaAlertsEnglishChannel), which provides real-time alerts about rocket sirens (Red Alerts) in Israel. The channel broadcasts information about alerts, including affected regions, cities, and timestamps. The dataset consists of extracted historical messages from this channel for further analysis and visualization.

In [ ]:
# Path to the folder containing files
path_to_files = r"C:\Users\Vera\Documents\DA Practicum\work files\datasets\cumta"

# Create an empty list to store data
data = []

# Iterate over all files with the prefix 'messages'
for file in os.listdir(path_to_files):
    if file.startswith('messages') and file.endswith('.html'):
        # Open the file in UTF-8 encoding
        with open(os.path.join(path_to_files, file), 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')
            
            # Process messages and find associated dates
            for msg in soup.find_all('div', class_='text'):
                # Find the nearest previous date element
                date = msg.find_previous('div', class_='pull_right date details')
                data.append({
                    'date': date['title'] if date and 'title' in date.attrs else None,
                    'text': msg.text.strip()
                })

# Convert the collected data into a DataFrame
raw_df = pd.DataFrame(data)

raw_df


#  Data preprocessing
## Creating a general dataset

In [ ]:
# Convert the 'date' column to datetime format with dayfirst=True
raw_df['date'] = pd.to_datetime(raw_df['date'], dayfirst=True)

# Sort the DataFrame by date in ascending order
raw_df = raw_df.sort_values(by='date', ascending=True).reset_index(drop=True)

# Checking the minimum and maximum dates in the dataset
print(f"Minimum date: {raw_df['date'].min()}")
print(f"Maximum date: {raw_df['date'].max()}")


In [ ]:
# Check
raw_df

In [ ]:
# Delete rows where the value in the date column is NaT
raw_df = raw_df.dropna(subset=['date']).reset_index(drop=True)
raw_df

In [ ]:
# Display general information 
raw_df.info()

In [ ]:
# --- Hidden because of the large size ---
# Let's see the whole dataframe
#raw_df.to_string()

In [ ]:
# A function for determining the type of threat
def extract_threat_type(text):
    if text.startswith("Red Alert") or text.startswith("An alert"):
        return "Red Alert"
    elif text.startswith("Unrecognized Aircraft"):
        return "Unrecognized Aircraft"
    elif text.startswith("An unrecognized aircraft"):
        return "Unrecognized Aircraft"
    elif text.startswith("Terrorist Infiltration"):
        return "Terrorist Infiltration"
    elif text.startswith("Earthquake"):
        return "Earthquake"
    elif text.startswith("Interception pieces"):
        return "Interception pieces"
    return None

# Removing update messages (starting with 'Good morning', 'Dear' or similar)
raw_df = raw_df[~raw_df['text'].str.lower().str.startswith(('good', 'starting', 
                                                            'dear', "home", 
                                                            'a new'))].reset_index(drop=True)

raw_df = raw_df.copy()
raw_df['threat_type'] = raw_df['text'].apply(extract_threat_type)


In [ ]:
# A function for extracting a region
def extract_region(text):
    match = re.search(r'at (.*?) \[\d{2}:\d{2}\]', text)
    return match.group(1).strip() if match else None

# A function for extracting time
def extract_time(text):
    match = re.search(r'\[(\d{2}:\d{2})\]', text)
    return match.group(1) if match else None

# Functions for extracting major cities and regional councils

def extract_major_cities(text):
    match = re.search(r'[Mm]ajor [Cc]ities: (.+?)(?:\|\||$)', text)
    return match.group(1).strip() if match else None

def extract_regional_councils(text):
    match = re.search(r'Regional Councils: (.+?)\|\|', text)
    return match.group(1).strip() if match else None

# Adding new columns
raw_df['region'] = raw_df['text'].apply(extract_region)
raw_df['major_cities'] = raw_df['text'].apply(extract_major_cities)
raw_df['regional_councils'] = raw_df['text'].apply(extract_regional_councils)

# Resetting the index for the resulting DataFrame
raw_df.reset_index(drop=True, inplace=True)

In [ ]:
# Check
raw_df

### 1 st october 2024 check
Let's check how strings are stored during very powerful attacks, when a lot of settlements were involved.

In [ ]:
# looking at the lines with the date of October 1, 2024 from 19 to 20 (the Iranian attack)
raw_df[
    (raw_df['date'] >= '2024-10-01 19:00:00') &
    (raw_df['date'] < '2024-10-01 20:00:00')
]

In [ ]:
# Checking the text in the 'text' column (row 5643)
raw_df.loc[5641, 'text']

In [ ]:
raw_df.loc[5642, 'text']

In [ ]:
raw_df.loc[5643, 'text']

In [ ]:
raw_df.loc[5644, 'text']

The information in the 'text' column gets truncated. The reason for this truncation is that a single message we want to process is split across multiple **div class="text"** blocks. The code processes only one **div class="text"** at a time without combining them into a single message. As a result, the message text is incomplete, and subsequent parts are saved as separate rows.

Rows that start with messages like "• Center Negev - Dvira Junction," have None in the threat_type, region, and time columns but actually belong to the previous message. We can process these rows by taking the threat_type from the previous message. If the text starts with "• ", we append this text to the text field of the previous row and then delete the current row.

In [ ]:
# Sort indices in descending order to avoid conflicts when modifying rows
for idx in sorted(raw_df[raw_df['text'].str.match(r"\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}:|• ", 
                                                  na=False)].index, reverse=True):
    # Check if the previous row exists
    if idx - 1 in raw_df.index:
        # Append the current text to the 'text' of the previous row
        raw_df.loc[idx - 1, 'text'] += f" {raw_df.loc[idx, 'text']}"
        # Drop the current row
        raw_df.drop(index=idx, inplace=True)

# Reset the index after modifications
raw_df.reset_index(drop=True, inplace=True)

In [ ]:
# Checking problematic rows (1 st october 2024, 19.00-20.00)
raw_df[
    (raw_df['date'] >= '2024-10-01 19:00:00') &
    (raw_df['date'] < '2024-10-01 20:00:00')
]

In [ ]:
# Check problematic srting
raw_df.loc[5626, 'text']

Now these strings are OK.

In [ ]:
# Merge rows where 'text' starts with '||' into the previous row
for idx in sorted(
    raw_df[raw_df['text'].str.startswith('||', na=False)].index, 
    reverse=True
):
    if idx - 1 in raw_df.index:
        raw_df.loc[idx - 1, 'major_cities'] = (
            f"{raw_df.loc[idx - 1, 'major_cities']} {raw_df.loc[idx, 'major_cities']}".strip()
        )
        raw_df.loc[idx - 1, 'regional_councils'] = (
            f"{raw_df.loc[idx - 1, 'regional_councils']} {raw_df.loc[idx, 'regional_councils']}".strip()
        )
        raw_df.drop(index=idx, inplace=True)


# Reset the index after modifications
raw_df.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
raw_df


Everything is fine now

In [ ]:
raw_df.info()

In [ ]:
# Checking how useful the "major_cities" column is
print('Number of major cities:', raw_df['major_cities'].nunique())
raw_df['major_cities'].unique()

In [ ]:
raw_df['major_cities'].value_counts().to_frame()

Let's see how useful the 'major_cities' and 'regional_councils' columns are.

In [ ]:
# Count the number of values that are either "None" (as a string) or missing (NaN, None)
none_count = raw_df['major_cities'].apply(lambda x: x == "None" or pd.isna(x)).sum()
none_count_rc = raw_df['regional_councils'].apply(lambda x: x == "None" or pd.isna(x)).sum()

print(f"Number of rows with 'None' or missing values in major_cities: {none_count}")
print(f"Number of rows with 'None' or missing values in regional_councils: {none_count_rc}")

There are too many rows with missing values, meaning the columns are not very useful. I will not use it in the future.

**Raw_df Processing Summary**  

- **Extracted** data from **Telegram messages**.  
- **Removed** irrelevant rows (NaT dates, update messages).  
- **Fixed truncated messages** caused by split `div class="text"` blocks.  
- **Merged fragmented rows** (e.g., those starting with "•") into previous messages.  
- **Analyzed missing values**:  
  - `major_cities`: **4250 missing**  
  - `regional_councils`: **6613 missing**  
  - **Decided to exclude these columns** due to high data loss.  

The dataset is now cleaned, structured, and ready for further analysis.

## Creating a dataset with an indication of detailed localities

In [ ]:
def extract_alerts(text):
    alerts = []
    
    # Regex pattern for datetime in the format "DD/MM/YYYY HH:MM:SS:"
    date_pattern = re.compile(r'(\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}):')
    
    # Find the first date occurrence and remove content before it
    first_date_match = date_pattern.search(text)
    if not first_date_match:
        return alerts  # If no date found, return an empty list
    text = text[first_date_match.start():]
    
    # Remove any content after the first occurrence of "||"
    if "||" in text:
        text = text.split("||")[0]
    
    # Regex to split the text into blocks (each block starts with a datetime)
    block_pattern = re.compile(
        r'(\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}):'
        r'(.*?)(?=(\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}:)|$)',
        re.DOTALL
    )
    
    # Iterate over each alert block
    for block_match in block_pattern.finditer(text):
        dt = block_match.group(1).strip()         # Extracted datetime
        block_text = block_match.group(2).strip()   # Text following the datetime
        
        # Split block text using bullet (•)
        bullet_parts = [
            part.strip() for part in re.split(r'\u2022', block_text)
            if part.strip()
        ]
        
        for part in bullet_parts:
            # Remove unwanted phrases such as "Click here to open an interactive map"
            # and "Sent by @CumtaAlertsEnglishChannel"
            part = re.sub(
                r'(Click here to open an interactive map|Sent by @CumtaAlertsEnglishChannel).*$', 
                '', part
            ).strip()
            
            # Determine if we use a dash (-) or a colon (:) as the separator
            dash_pos = part.find('-')
            colon_pos = part.find(':')
            
            if dash_pos != -1 and (colon_pos == -1 or dash_pos < colon_pos):
                # Case 1: "Region - locality1, locality2, ..."
                region_part, localities_part = part.split('-', 1)
            elif colon_pos != -1:
                # Case 2: "Region: locality1, locality2, ..."
                region_part, localities_part = part.split(':', 1)
            else:
                continue  # If neither separator exists, skip this entry
            
            # Clean up the extracted region name
            region = region_part.strip()
            region = re.sub(
                r'(Click here to open an interactive map|Sent by @CumtaAlertsEnglishChannel).*$', 
                '', region
            ).strip()
            region = region.rstrip(':- ').strip()
            
            # Clean localities
            localities_part = re.sub(
                r'(Click here to open an interactive map|Sent by @CumtaAlertsEnglishChannel).*$', 
                '', localities_part
            ).strip()
            localities = [loc.strip() for loc in localities_part.split(',') if loc.strip()]
            
            # If a region name is too generic, treat it as part of the locality
            generic_regions = {"Dan Area", "Lachish Area", "HaSharon Region", "Negev Region"}
            if region in generic_regions:
                for loc in localities:
                    alerts.append({
                        'datetime': dt,
                        'region': loc,
                        'locality': region
                    })  # Swap region/locality
            else:
                for loc in localities:
                    alerts.append({
                        'datetime': dt,
                        'region': region,
                        'locality': loc
                    })  # Normal case
    
    return alerts

# Process each row in raw_df to extract alerts
all_rows = []
for _, row in raw_df.iterrows():
    text = row['text']
    threat_type = row['threat_type']
    alerts = extract_alerts(text)
    for alert in alerts:
        alert['threat_type'] = threat_type
        all_rows.append(alert)

# Create a new DataFrame with detailed alerts
detailed_df = pd.DataFrame(all_rows)

In [ ]:
detailed_df

In [ ]:
# looking at the lines with the date of October 1, 2024 from 19 to 20 (the Iranian attack)
detailed_df[
    (detailed_df['datetime'] >= '01/10/2024 19:50:00') &
    (detailed_df['datetime'] < '01/10/2024 20:00:00')
].tail(20)

In [ ]:
detailed_df.info()

In [ ]:
detailed_df['datetime'] = pd.to_datetime(detailed_df['datetime'], dayfirst=True, errors='coerce')

In [ ]:
# Let's check what the contents of the first row of the column 'text' look like.
raw_df.loc[0, 'text']

In [ ]:
# Check
detailed_df.head(20)

### ⚠ Checking the distribution of data by year

In [ ]:
# Convert the 'time' column to datetime format
detailed_df['datetime'] = pd.to_datetime(detailed_df['datetime'], 
                                         errors='coerce', format='%d/%m/%Y %H:%M:%S')

# Extract the year from the datetime column
detailed_df['year'] = detailed_df['datetime'].dt.year

# Count the number of warnings per year
warnings_by_year = detailed_df['year'].value_counts().sort_index()

# Calculate the percentage each year represents
percentages = (warnings_by_year / warnings_by_year.sum()) * 100

# Combine counts and percentages into a DataFrame
result_df = pd.DataFrame({
    'Count': warnings_by_year,
    'Percentage': percentages.round(2).astype(str) + '%'
})

# Display the result
print('Distribution of data by year in detailed_df:')
result_df

In [ ]:
# Use the existing 'date' column which already contains datetime values
raw_df['year'] = raw_df['date'].dt.year

# Count the number of alerts per year
warnings_by_year = raw_df['year'].value_counts().sort_index()

# Calculate the percentage each year represents
percentages = (warnings_by_year / warnings_by_year.sum()) * 100

# Combine counts and percentages into a DataFrame
raw_result_df = pd.DataFrame({
    'Count': warnings_by_year,
    'Percentage': percentages.round(2).astype(str) + '%'  # Append '%' to the rounded percentages
})

# Display the result
print('Distribution of data by year in raw_df:')
raw_result_df


⚠ WARNING ⚠ Issue Identified:

There is a noticeable discrepancy in the data distribution by year between the two datasets, `detailed_df` and `raw_df`.

- In **`detailed_df`**, the year **2021** accounts for **35.09%** of the dataset.
- In **`raw_df`**, the year **2021** represents only **14.06%** of the dataset.

Also, all our information initially consists of 15 files. The alarm lines for 2021 take up slightly less than 2 files out of 15, that is, about 13-14%


### Checking threat types

In [ ]:
print('Number of threat types:', detailed_df['threat_type'].nunique())
detailed_df['threat_type'].unique()

### Checking the 'region' and 'locality' columns

In [ ]:
print('Number of regions:', detailed_df['region'].nunique())
detailed_df['region'].unique()

In [ ]:
print('Number of localities:', detailed_df['locality'].nunique())
print(detailed_df['locality'].unique())

In [ ]:
# --- Hidden because of the large size ---
# Check how the localities were divided to fix the function, if necessary.
#detailed_df['locality'].to_string()

### Checking localities and regions in Hebrew

In [ ]:
# Checking if there are names of localities in Hebrew.
detailed_df[detailed_df['locality'].str.contains(r'[\u0590-\u05FF]', na=False)]

In [ ]:
# Display all the localities in Hebrew with the number of times
detailed_df[detailed_df['locality'].str.contains(r'[\u0590-\u05FF]', 
                                                 na=False)].value_counts().to_frame()

In [ ]:
translation_dict = {
    "צפת": "Safed",
    "נירים": "Nirim",
    "אשדוד": "Ashdod",
    "איבים": "Ibim",
    "שדרות": "Sderot",
    "קריית שמונה": "Kiryat Shmona",
    "ניר עם": "Nir Am",
    "לב החולה": "Lev Ha-Hula",
    "פרדס חנה": "Pardes Hanna - Karkur",
    "כוחלה מכחול": "Kokhav Michael",
    "שוהם": "Shoham",
    "כרכור": "Karkur",
    "כפר נחום": "Kfar Naḥum",
    "מצוק עורבים": "Orevim Cliff",
    "איירפורט סיטי": "Airport City",
    "אזור תעשייה רמת גן": "Ramat Gan Industrial Zone",
    "רמת טראמפ": "Ramat Trump",
    "רכסים נהר הירדן": "Jordan River Terraces",
    "אזור תעשייה תעשייה רמת גן": "Ramat Gan Industrial Area",
    "חוות אירוח גורן": "Goren Guest Farm",
    "תחנה": "Station",
    "מוזיאון כוכבים רעים": "Bad Stars Museum",
    "מלון אחוזת ירדן": "Jordan Estate Hotel",
    "מיני ישראל": "Mini Israel",
    "נאות קדומים": "Neot Kedumim",
    "חניון הנגב מהר": "Negev Fast Parking Lot",
    "מלון מונזון": "Monzon Hotel",
    "יהוד מונוסון": "Yehud Monoson",
    "גני יהודה": "Ganei Yehuda",
    "רמת טראמפ": "Ramat Trump",
    "צוק עורבים": "Ravens' Cliff",
    "מכון תבנית מהר": "Fast Template Institute",
    "מבטחים עמיעוז ישע": "Mivtachim Ami'oz Yesha",
    "רפטינג נהר הירדן": "Jordan River Rafting",
    "אזור תעשייה רגמ": "Regem Industrial Zone",
    "חניון הנתיב מהיר": "Fast Lane Parking Lot",
    "מודיעין מכבים רעות": "Modiin Maccabim Reut",
    "נילי": "Nili",
    "טבחה": "Tabgha",
    "איזור תעשייה מילואות צפון": "Miluot North Industrial Zone"
}



# Function to replace entire string if it contains a Hebrew name
def replace_entire_entry(text, translation_dict):
    if pd.isna(text):  # Handle NaN values safely
        return text
    for hebrew, english in translation_dict.items():
        if hebrew in text:  # If any Hebrew name appears in the text
            return english  # Replace the entire text with the English name
    return text  # Otherwise, keep the original value

# Apply the function to 'locality' and 'region' columns
detailed_df['locality'] = (
    detailed_df['locality']
    .astype(str)
    .str.strip()
    .apply(lambda x: replace_entire_entry(x, translation_dict))
)

detailed_df['region'] = (
    detailed_df['region']
    .astype(str)
    .str.strip()
    .apply(lambda x: replace_entire_entry(x, translation_dict))
)


In [ ]:
# Display all the localities in Hebrew with the number of times
detailed_df[detailed_df['locality'].str.contains(r'[\u0590-\u05FF]', 
                                                 na=False)].value_counts().to_frame()

In [ ]:
# List of Hebrew words to be removed from the 'locality' column
words_to_remove = ["טו", "יב", "יז", "מרינה", "יא", "סיט"]

# Filter the DataFrame to exclude rows where 'locality' contains any of the specified words
detailed_df = detailed_df[~detailed_df['locality'].isin(words_to_remove)]

# Display all the localities in Hebrew with the number of times
detailed_df[detailed_df['locality'].str.contains(r'[\u0590-\u05FF]', 
                                                 na=False)].value_counts().to_frame()

I have translated all the Hebrew names into their English versions. 

In [ ]:
# Check again
detailed_df['region'].unique()

### Checking the number of types of city names

We have relatively large cities that are divided into zones. Let's check how many such zones there are

In [ ]:
# List of major cities
major_cities = [
    "Eilat", "Modi'in", "Hadera", "Caesarea", "Ramat Gan", "Haifa", 
    "Beer Sheva", "Netanya", "Rishon LeZion", "Tel Aviv", "Ashdod", "Herzeliya", "Jerusalem", 
    "Beit Shemesh", "Ashkelon", "Rehovot", "Lod", "Ramla", "Holon", "Bat Yam", "Kfar Saba", 
    "Petach Tikva", "Tiberias", "Nahariya", "Safed", "Kiryat Shmona", "Acre",
    "Ma'ale Adumim", "Ariel", "Nazareth", "Atlit", "Sderot", "Ofakim", 
    "Dimona", "Yavne", "Kiryat Gat", "Kiryat Malakhi", "Migdal HaEmek", "Or Akiva",
    "Be'er Ya'akov", "Givatayim", "Yokneam Illit", "Tirat Carmel", "Karmiel", "Arad",
    "Ma'alot-Tarshiha", "Kiryat Ata", "Kiryat Bialik", "Kiryat Yam", "Kiryat Motzkin",
    "Nesher", "Afula", "Bnei Brak", "Kiryat Ono", "Or Yehuda", "Ramat HaSharon", 
    "El'ad", "Ganei Tikva", "Giv'at Shmuel", "Hod Hasharon", "Kafr Qasim", "Kfar Yona", 
    "Ness Ziona", "Qalansawe", "Ra'anana", "Rosh HaAyin", "Tayibe", "Tira", 
    "Yehud-Monosson", "Pardes Hanna-Karkur"
]

def get_localities_by_city(df, major_cities, column='locality'):
    """
    Function to retrieve unique localities for each major city.
    
    :param df: DataFrame containing the data.
    :param major_cities: List of major cities to check.
    :param column: Name of the column to search for localities (default is 'locality').
    :return: Dictionary where the key is the city and the value is a list of unique localities.
    """
    city_localities = {}
    for city in major_cities:
        # Filter the DataFrame and extract unique localities for the current city
        localities = df[df[column].str.contains(city, case=False, na=False)][column].unique()
        city_localities[city] = localities.tolist()  # Convert to a list for easier handling
        print(f"City: {city}, Localities: {city_localities[city]}")  # Debug output
        print("=" * 40)  # Separator line
    return city_localities

# Call the function
localities_by_city = get_localities_by_city(detailed_df, major_cities)

Let's create a function that retrieves unique localities for a given city from a DataFrame. This function will filter the data based on the city name, extract distinct localities from the specified column, and return them as a list. Additionally, it will display the results in a structured format for better readability. This can be useful for analyzing geographic data and understanding the distribution of localities within a city.

In [ ]:
def get_localities_by_city(df, city, column='locality'):
    """
    Function to retrieve unique localities for a specific city.

    :param df: DataFrame containing the data.
    :param city: The city to check (as a string).
    :param column: Name of the column to search for localities (default is 'locality').
    :return: List of unique localities for the specified city.
    """
    # Filter the DataFrame and extract unique localities for the city
    localities = df[df[column].str.contains(city, case=False, na=False)][column].unique()
    
    # Convert to list for easier handling
    localities_list = localities.tolist()
    
    # Print result for the city
    print(f"City: {city}")
    print("=" * 40)  # Separator line
    print("Localities:")
    if localities_list:
        print(", ".join(localities_list))
    else:
        print("No localities found")
    print("=" * 40)  # Separator line

    return localities_list


⚠ The following check of the city zones was done using the ChatGPT to speed up and may contain errors. ⚠

In [ ]:
get_localities_by_city(detailed_df, "Tel Aviv")

The following entries are most likely part of Tel Aviv city in its classic sense: 'Tel Aviv (South West)',  'Tel Aviv (North)' 
 'Tel Aviv (Central) 
 'Tel Aviv (South East ',
 'Tel Aviv - City Ce.
te**Jaffa (Yafo)** is a distinct historic area often treated separately due to its unique cultural and historical significance. But let's classify this area as Tel Aviv, since it is officially part of the Tel Aviv-Yafo municipality.
r']



In [ ]:
get_localities_by_city(detailed_df, "Jerusalem")

Almost all entries from the list are officially part of Jerusalem, except 'Jerusalem - North and Alonim', which is potentially ambiguous, because "Alonim" may refer to areas adjacent to Jerusalem or informal names not officially within its boundaries. However, it should be included in Jerusalem for consistency, as such entries are often culturally or geographically associated with the city, ensuring unified classification and avoiding fragmentation in the dataset.

In [ ]:
get_localities_by_city(detailed_df, "Ashdod")

**'Hatzor Ashdod'** is not a part of Ashdod proper. The rest of the zones are officially part of it.

In [ ]:
get_localities_by_city(detailed_df, "Ashkelon")

⚠There are differing opinions in various sources regarding whether the industrial zones are officially part of Ashkelon. However, given their close proximity to residential areas, I assume their inclusion within the city for the purpose of analysis.⚠


In [ ]:
get_localities_by_city(detailed_df, "Modi'in")

The following zones are officially part of Modi'in-Maccabim-Re'ut (the city): **Modi'in - Ishpro Center**, **Modi'in - Ligad Center**, **Modi'in-Maccabim-Re'ut**, **Modi'in Maccabim Re'ut**.
The remaining entries, such as **Hevel Modi'in**, **Modi'in Illit**, and **industrial zones**, refer to nearby regional councils or separate municipalities and are not officially part of the city.

In [ ]:
get_localities_by_city(detailed_df, "Ramat Gan")

The entry "Ramat Gan - Ramat Ef'al & Tel Hashomer" partially refers to areas outside the city (e.g., Tel Hashomer, which belongs to Kiryat Ono).  The rest of the zones are officially part of the city.

In [ ]:
get_localities_by_city(detailed_df, "Haifa")

The entry **"Haifa - Kiryat Haim & Kiryat Shmuel"** refers to neighborhoods that are administratively part of Haifa but are often considered distinct communities within the city. Other zones are officially part of Haifa.

There are such inscriptions in the dataset in the areas of Haifa
• Menashe - Haifa - Ramot HaCarmel and Neveh Sha'anan, Haifa - Carmel, Hadar and Downtown Lower City.

That is, we have 'Hadar and Downtown Lower City'
We should divide them into 'Haifa - Hadar' and 'Haifa - Downtown Lower City'

⚠ Also, there is a special case with the name Carmel ⚠

In [ ]:
get_localities_by_city(detailed_df, "Carmel")

df[df['locality'] == 'Ashdod']Issue Explanation:⚠

There is a **naming conflict** with the term **"Carmel"** in your dataset. **Carmel** can refer to multiple localities, and it’s important to differentiate between them to avoid misclassification.


The following **localities are officially neighborhoods or areas within Haifa**:

1. **Haifa - Carmel and Lower City**  
2. **Haifa - Ramot HaCarmel and Neveh Sha'anan**  
3. **Haifa - Carmel**

These names represent **recognized neighborhoods** in Haifa, situated on **Mount Carmel**, which is a central geographical feature of the city.



These localities are **close to Haifa** and associated with the **Carmel region** but are **not administratively part of the city**:

1. **Tirat Carmel** – A separate city located just south of Haifa.
2. **Carmel Forest Spa Resort** – A famous resort located in the Carmel mountain range near Haifa.
3. **Mevo Carmel Industrial Zone** – An industrial area near the Carmel region but not within Haifa’s official boundaries.
4. **Hof HaCarmel** – Refers to the **Carmel Coast Regional Council**, which is a separate administrative region near Haifa.


Some localities contain the name "Carmel" but are **not related to Haifa**:

1. **Yehuda - Carmel**  This is an **independent locality** located in the **Har Hevron (Mount Hebron) Regional Council** in the southern part of Israel, far from Haifa.

2. **Yearot HaCarmel, Geva Carmel, Ein Carmel**  These are **villages or settlements** in the broader Carmel region but **not part of Haifa's municipal jurisdiction**.


In [ ]:
get_localities_by_city(detailed_df, 'Acre')

Acre, but there are also 'Akko', 'Acco'

In [ ]:
# Filter rows where 'locality' contains 'Akko' or 'Acco'
akko_variants = detailed_df[detailed_df['locality'].str.contains(r'Akko|Acco', case=False, na=False)]

# Display unique values
unique_akko_variants = akko_variants['locality'].unique()

# Print results
print("Unique locality values containing 'Akko' or 'Acco':")
for variant in unique_akko_variants:
    print(variant)


In [ ]:
# Replace 'Acco' and 'Akko' with 'Acre' in the 'locality' column
detailed_df.loc[:, 'locality'] = detailed_df['locality'].str.replace(r'Acco|Akko', 'Acre', case=False, regex=True)

In [ ]:
get_localities_by_city(detailed_df, "Hadera")

All the listed zones are officially part of Hadera

In [ ]:
get_localities_by_city(detailed_df, "Caesarea")

The following zones are officially part of Caesarea: Caesarea, Caesarea Marine Center
The Caesarea Industrial Zone is a nearby industrial area but is not part of the residential or municipal core of Caesarea. But we will designate it as a part of the city, because we have done the same for the industrial zones of other cities.

In [ ]:
get_localities_by_city(detailed_df, "Herzeliya")

All the listed zones are officially part of Herzeliya.

There is also 'Herzliya'

In [ ]:
# Replace 'Herzliya' with 'Herzeliya' in the 'locality' column
detailed_df.loc[:, 'locality'] = detailed_df['locality'].str.replace(r'Herzliya', 'Herzeliya', case=False, regex=True)


In [ ]:
get_localities_by_city(detailed_df, "Rehovot")

All the listed zones are officially part of Rehovot

In [ ]:
get_localities_by_city(detailed_df, "Ramla")

Both of the mentioned localities are part of the city of Ramla:

**Ramla** – This is the city itself, located in the central district of Israel.

**Nesher Industrial Zone (Ramla)** – This is an industrial zone located within Ramla. It is named after Nesher, one of Israel’s leading cement manufacturers, and is situated within the city limits.

In [ ]:
get_localities_by_city(detailed_df, "Nahariya")

Both Nahariya and Nahariya Cemetery are part of the city:

**Nahariya** – This is the city itself, located in the Northern District of Israel.

**Nahariya Cemetery** – This is the city's cemetery, situated within the boundaries of Nahariya

In [ ]:
get_localities_by_city(detailed_df, "Safed")

Belongs to the city:
- **Safed** – Refers to the city itself, located in the Northern District of Israel.  
- **Safed - City** – Specifically refers to the central urban area of Safed.  
- **Safed - Nof ha-Kinneret** – A neighborhood or area within the municipal boundaries of Safed.  
- **Safed - 'Akbara** – Officially part of the city of Safed since 1977. 'Akbara is administered as a neighborhood within the city's municipal boundaries.  

So, **all** the listed localities belong to **Safed**.

In [ ]:
get_localities_by_city(detailed_df, "Nazareth")

Belongs to the city:
- **Nazareth** – This is the city itself, located in the Northern District of Israel. It is the largest Arab city in the country and has its own independent municipality.

Does **not** officially belong to the city:
- **Nof HaGalil (Nazareth Illit)** – This is a separate city with its own municipality. Although it was originally established as a Jewish suburb of Nazareth, it became an independent city and officially changed its name from *Nazareth Illit* to *Nof HaGalil* in 2019.

In [ ]:
get_localities_by_city(detailed_df, "Dimona")

**Both Dimona and Dimona Industrial Zone officially belong to the city of Dimona.**

- **Dimona**  
This is the city itself, located in the Southern District of Israel. It has its own municipality and administrative boundaries.

- **Dimona Industrial Zone**  
This industrial area is officially part of Dimona’s municipal jurisdiction. It serves as the city’s hub for industrial and economic activities but is distinct from the residential areas.

In [ ]:
get_localities_by_city(detailed_df, "Yavne")

Only **Yavne** and the **Yavne Industrial Zone** are officially part of the city of Yavne. The other localities are separate entities and do not fall under Yavne's municipal jurisdiction. 

**Officially Part of Yavne:**

- **Yavne**: The city itself, located in central Israel.

- **Yavne Industrial Zone**: An industrial area within Yavne's municipal boundaries, serving as a hub for the city's industrial and economic activities.

**Not Officially Part of Yavne:**

- **Gan Yavne**: A local council situated east of Ashdod, operating as an independent municipality separate from Yavne.

- **Yavne Region Industries**: This term likely refers to industrial areas in the broader Yavne region but not necessarily within Yavne's city limits.

- **Hevel Yavne**: A regional council encompassing several communities in the area surrounding Yavne, but not part of the city itself.

- **Kvutzat Yavne**: A religious kibbutz located near Yavne, falling under the jurisdiction of the Hevel Yavne Regional Council. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kvutzat_Yavne?utm_source=chatgpt.com))

- **Kerem Yavneh (Kerem BeYavne)**: A yeshiva and youth village adjacent to Kvutzat Yavne, also under the Hevel Yavne Regional Council. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Yeshivat_Kerem_B%27Yavneh?utm_source=chatgpt.com))

- **Yavne'el**: A moshava in northern Israel, not geographically or administratively connected to the city of Yavne.

- **Sheni LeYavne**: This term translates to "Second to Yavne" but does not correspond to a recognized locality within or near Yavne.


In [ ]:
get_localities_by_city(detailed_df, "Kiryat Gat")

Both **Kiryat Gat** and **Kiryat Gat - Industrial Zone** officially belong to the city of **Kiryat Gat**.

**Kiryat Gat**  
This is the city itself, located in the Southern District of Israel. It has its own municipality and serves as a regional center for the surrounding area.

**Kiryat Gat - Industrial Zone**  
This industrial zone is officially part of Kiryat Gat’s municipal jurisdiction. It includes major industrial facilities and tech companies, contributing significantly to the city’s economy.

In [ ]:
get_localities_by_city(detailed_df, "Yokneam Illit")

 Both **Yokneam Illit** and **Yokneam Illit Industrial Zone** are officially part of the city of **Yokneam Illit**.

**Yokneam Illit**   
This is the city itself, located in northern Israel at the base of the Carmel Mountains. It has its own municipality and is known for its thriving high-tech industry.

**Yokneam Illit Industrial Zone**   
This industrial area is officially part of Yokneam Illit’s municipal jurisdiction. It serves as a key hub for technological companies and industrial activities, contributing significantly to the city’s economy.



In [ ]:
get_localities_by_city(detailed_df, "Arad")

Only **Arad** is officially part of the city of **Arad**. The other localities mentioned are separate entities and do not fall under Arad's municipal jurisdiction. 

Officially Part of Arad:

- **Arad**: This is the city itself, located in the Southern District of Israel, on the border of the Negev and Judean Deserts. It has its own municipality and administrative boundaries. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Arad%2C_Israel?utm_source=chatgpt.com))

Not Officially Part of Arad:

- **Tel Arad**: An archaeological site situated approximately 10 kilometers west of the modern city of Arad. It features the remains of a fortified Canaanite city and Israelite fortresses. Tel Arad is a national park and is not within the municipal boundaries of Arad. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Tel_Arad?utm_source=chatgpt.com))

- **El Pura**: This locality is not widely recognized in available sources and does not appear to be officially associated with the city of Arad.

In [ ]:
get_localities_by_city(detailed_df, "Kiryat Bialik")

**The Kiryat Bialik Industrial Zone** is officially part of the city of **Kiryat Bialik**.

In [ ]:
get_localities_by_city(detailed_df, "Kiryat Bialik")

But there is also 'Kiryat Biyalik' in the dataset.

In [ ]:
# Replace 'Herzliya' with 'Herzeliya' in the 'locality' column
detailed_df.loc[:, 'locality'] = detailed_df['locality'].str.replace(r'Kiryat Biyalik', 'Kiryat Bialik', case=False, regex=True)

### Updating the dataset (dividing zones)

#### Haifa
That is, we have 'Hadar and Downtown Lower City'
We should divide them into 'Haifa - Hadar' and 'Haifa - Downtown Lower City'

In [ ]:
expanded_rows = []
for _, row in detailed_df.iterrows():
    if row['locality'] == 'Hadar and Downtown Lower City':
        localities = ['Haifa - Hadar', 'Haifa - Downtown Lower City']
    elif row['locality'] == 'Haifa - Carmel and Lower City':
        localities = ['Haifa - Carmel', 'Haifa - Downtown Lower City']
    elif row['locality'] == "Haifa - Ramot HaCarmel and Neveh Sha'anan":
        localities = ['Haifa - Ramot HaCarmel', "Haifa - Neveh Sha'anan"]
    else:
        localities = [row['locality']]
    
    for loc in localities:
        new_row = row.copy()
        new_row['locality'] = loc
        expanded_rows.append(new_row)

# Overwrite the original DataFrame
detailed_df = pd.DataFrame(expanded_rows).reset_index(drop=True)

detailed_df[detailed_df['locality'].str.contains('Haifa', case=False, na=False)]

In rare cases, in a text with localities, the city and zones are separated by commas, the line may be separated without specifying the city. For example, 'Haifa - Carmel, Hadar and Downtown Lower City'. It is worth checking how many rows in the dataset are separated in the case of Haifa.

In [ ]:
keywords = ['Carmel', 'Downtown Lower City', 'Lower City', 'West', 
            'Hadar', 'Bay', 'Ramot HaCarmel', "Neveh Sha'anan"]

# Create a dictionary to store the counts
counts = {key: detailed_df[detailed_df['locality'] == key].shape[0] for key in keywords}

# Print the counts
for key, count in counts.items():
    print(f"{key}: {count} rows")



In [ ]:
# Replace 'Carmel' with 'Haifa - Carmel' in the 'locality' column except 'region' == 'Yehuda'
detailed_df['locality'] = detailed_df.apply(
    lambda row: (
        'Haifa - Carmel' 
        if row['locality'] == 'Carmel' and row['region'] != 'Yehuda' 
        else row['locality']
    ), 
    axis=1
)


In [ ]:
# Check
get_localities_by_city(detailed_df, "Haifa")

In [ ]:
# Print the counts
for key, count in counts.items():
    print(f"{key}: {count} rows")

Nothing has changed for the name 'Carmel'.

#### Ashdod
In the city of Ashdod, neighborhoods are often listed after the city's name, separated by commas. In the DataFrame, these neighborhoods are named directly as:

Initially, we had these regions:
- `Ashdod - Alef, Bet, Dalet, Heh`
- `Ashdod - Yod Alef, Yod Bet, Tet Vav, Yod Zain, Ma*`
- `Ashdod - Gimmel, Vav, Zain` 
- `Ashdod - Het, Tet, Yod, Yod Gimmel, Yod Dalet, Te*`
- `Ashdod-11,12,15,17,Marine,City`

When creating detailed_df, these regions became separate rows in the dataframe, which could have skewed the distribution of the data.

We have to do the following:
- If we have 'Ashdod - Alef', we change it to 'Ashdod - Alef, Bet, Dalet, Heh'
- If 'Ashdod - Yod Alef', we change it to 'Ashdod - Yod Alef, Yod Bet, Tet Vav, Yod Zain, Ma*'
- If we have 'Ashdod - Gimmel', we change it to 'Ashdod - Gimmel, Vav, Zain'
- If we have 'Ashdod - Het', we change it to 'Ashdod - Het, Tet, Yod, Yod Gimmel, Yod Dalet, Te*'
- If we have 'Ashdod-11', we change it to 'Ashdod - 11, 12, 15, 17, Marine, City'

We also have strings from ashdod_zones =
'Alef', 'Bet', 'Dalet', 'Gimmel', 'Heh', 'Het', 'Tet',
'Yod', 'Yod Gimmel', 'Yod Dalet', 'Vav', 'Zain', 'Marine', 'City', '12', '15', '17',

If 'locality' contains a string from ashdod_zones, it should be removed.

In [ ]:
detailed_df[detailed_df['locality'] == 'Heh']

In [ ]:
# Dictionary for replacements
replacements = {
    'Ashdod - Alef': 'Ashdod - Alef, Bet, Dalet, Heh',
    'Ashdod - Yod Alef': 'Ashdod - Yod Alef, Yod Bet, Tet Vav, Yod Zain, Ma*',
    'Ashdod - Gimmel': 'Ashdod - Gimmel, Vav, Zain',
    'Ashdod - Het': 'Ashdod - Het, Tet, Yod, Yod Gimmel, Yod Dalet, Te*',
    'Ashdod-11': 'Ashdod - 11, 12, 15, 17, Marine, City'
}

# Apply replacements
detailed_df['locality'] = detailed_df['locality'].replace(replacements)

# List of Ashdod zones to be removed
ashdod_zones = [
    'Alef', 'Bet', 'Dalet', 'Gimmel', 'Heh', 'Het', 'Tet', 'Ma*', 'Te*',
    'Yod', 'Yod Gimmel', 'Yod Dalet', 'Vav', 'Zain', 'Marine', 'City', '12', '15', '17'
]

# clean locality values
detailed_df = detailed_df[~detailed_df['locality'].isin(ashdod_zones)]

In [ ]:
detailed_df[detailed_df['locality'] == 'Heh']

In [ ]:
detailed_df[detailed_df['locality'].str.contains('Ashdod', case=False, na=False)]


In [ ]:
# Check
get_localities_by_city(detailed_df, "Ashdod")

In [ ]:
detailed_df.info()

In [ ]:
detailed_df

### ⚠ Handling Data Skew and Duplication in 2021

Let's try to find the reasons for data skew.

In [ ]:
# Separating the records for 2021 into a new dataset
detailed_df_2021 = detailed_df[detailed_df['year'] == 2021]

In [ ]:
detailed_df_2021.info()

In [ ]:
# Looking at the first 60 entries
detailed_df_2021.head(60)

Several lines with the same name of a locality can go one after the other. For example, Netiv HaAssara, Kissufim

In [ ]:
detailed_df_2021[detailed_df_2021['locality'] == 'Kissufim'].head(10)

In [ ]:
# Filtering the data for October 7, 2023 (massive attacks)
detailed_df[detailed_df['datetime'].dt.date == pd.to_datetime('2023-10-07').date()].head(50)

In 2023, the duplication issue was fixed, and each alert is now uniquely generated, with no duplicate segments.

In [ ]:
# Checking the unique names of localities
detailed_df_2021['locality'].unique()

Localities contain 'locality' == " in their names. Delete them from the entire dataset

In [ ]:
# Counting the number of such rows in the dataset
detailed_df[detailed_df['locality'] == ''].shape[0]

In [ ]:
# Cleaning data from rows with 'locality' == "
detailed_df = detailed_df[detailed_df['locality'] != '']

A post dated May 20, 2021, was found in the message files, containing information that
- Fixed a bug that caused duplicate alerts.

This means that previously, alarm alerts were duplicated, which led to incorrect distribution in the datas


⚠⚠⚠ Before May 20, 2021, the dataset contained duplicate alert blocks—identical lists of localities and timestamps were repeated within individual alerts. A May 20, 2021 update to Cumta’s Android app fixed this bug, but the duplicates skewed the yearly distribution of alerts.⚠⚠⚠

In [ ]:
# Filter rows before May 20, 2021
df_before_may20 = detailed_df[detailed_df['datetime'] < pd.Timestamp('2021-05-20')].copy()

# Create a unique key for each alert (based on 'datetime', 'region', and 'locality' columns)
df_before_may20['unique_key'] = (
    df_before_may20['datetime'].astype(str).str.strip() + '_' +
    df_before_may20['region'].astype(str).str.strip() + '_' +
    df_before_may20['locality'].astype(str).str.strip()
)

# Remove duplicates based on the unique key
df_before_may20_clean = df_before_may20.drop_duplicates(subset='unique_key').copy()

# Drop the unique key column if it is no longer needed
df_before_may20_clean.drop(columns=['unique_key'], inplace=True)

# If needed, merge the cleaned data with the remaining dataset (rows from May 20, 2021, and later)
detailed_df = pd.concat([
    df_before_may20_clean,
    detailed_df[detailed_df['datetime'] >= pd.Timestamp('2021-05-20')]
])


The number of rows has decreased significantly

In [ ]:
detailed_df[detailed_df['year'] == 2021].head(60)

There are no more duplicates

In [ ]:
# Convert the 'datetime' column to datetime format
detailed_df['datetime'] = pd.to_datetime(detailed_df['datetime'], 
                                         errors='coerce', format='%d/%m/%Y %H:%M:%S')

# Extract the year from the 'datetime' column
detailed_df['year'] = detailed_df['datetime'].dt.year

# Count the number of alerts per year
warnings_by_year_clean = detailed_df['year'].value_counts().sort_index()

# Calculate the percentage distribution per year
percentages_clean = (warnings_by_year_clean / warnings_by_year_clean.sum()) * 100

# Combine the counts and percentage distribution into a DataFrame
result_df_clean = pd.DataFrame({
    'Count': warnings_by_year_clean,
    'Percentage': percentages_clean.round(2).astype(str) + '%'
})

# Display the result
print('Distribution of data by year in detailed_df_clean:')
result_df_clean


🎉🎉🎉The distribution by year has become similar to the original one🎉🎉🎉

### Processing of the 'region' column

In [ ]:
# Check again
detailed_df['region'].unique()

In [ ]:
# Remove ' Area' from the 'region' column if it exists
detailed_df['region'] = detailed_df['region'].str.replace(' Area', '', regex=False).str.strip()
detailed_df['region'].unique()

In [ ]:
# Replace the incorrect region name with the corrected version
detailed_df['region'] = detailed_df['region'].replace('Milouot Industrial\xa0Zone\xa0North', 
                                                      'Milouot Industrial Zone North')

### Adding the 'district' column

In [ ]:
# List of major cities with their corresponding districts
city_to_district = {
    "Tel Aviv": "Tel Aviv District",
    "Dan Area": "Tel Aviv District",
    "Ramat Gan": "Tel Aviv District",
    "Herzeliya": "Tel Aviv District",
    "Holon": "Tel Aviv District",
    "Bat Yam": "Tel Aviv District",
    "Bnei Brak": "Tel Aviv District",
    "Givatayim": "Tel Aviv District",
    "Kiryat Ono": "Tel Aviv District",
    "Or Yehuda": "Tel Aviv District",
    "Ramat HaSharon": "Tel Aviv District",
    "Rishon LeZion": "Central District",
    "Netanya": "Central District",
    "Rehovot": "Central District",
    "Petah Tikva": "Central District",
    "Lod": "Central District",
    "Ramla": "Central District",
    "Kfar Saba": "Central District",
    "Yavne": "Central District",
    "Modi'in-Maccabim-Re'ut": "Central District",
    "Modi'in": "Central District",
    "Be'er Ya'akov": "Central District",
    "El'ad": "Central District",
    "Ganei Tikva": "Central District",
    "Giv'at Shmuel": "Central District",
    "Hod Hasharon": "Central District",
    "Kafr Qasim": "Central District",
    "Kfar Yona": "Central District",
    "Ness Ziona": "Central District",
    "Qalansawe": "Central District",
    "Ra'anana": "Central District",
    "Rosh HaAyin": "Central District",
    "Tayibe": "Central District",
    "Tira": "Central District",
    "Yehud-Monosson": "Central District",
    "Haifa": "Haifa District",
    "Hadera": "Haifa District",
    "Caesarea": "Haifa District",
    "Nesher": "Haifa District",
    "Or Akiva": "Haifa District",
    "Tirat Carmel": "Haifa District",
    "Kiryat Ata": "Haifa District",
    "Kiryat Bialik": "Haifa District",
    "Kiryat Yam": "Haifa District",
    "Kiryat Motzkin": "Haifa District",
    "Pardes Hanna-Karkur": "Haifa District",
    "Jerusalem": "Jerusalem District",
    "Beit Shemesh": "Jerusalem District",
    "Ma'ale Adumim": "Jerusalem District",
    "Nazareth": "Northern District",
    "Acre": "Northern District",
    "Gan Yavne": "Central District",
    "Dead Sea Factories": "Southern District",
    "Dead Sea Industries": "Southern District",
    "Lakhish": "Southern District",
    "Atlit": "Haifa District",
    "Kiryat Malachi": "Southern District",
    "Gdera": "Central District",
    "Bnei Darom": "Southern District",
    "Sdeh Yoav": "Southern District",
    "Palmachim": "Central District",
    "Tiberias": "Northern District",
    "Nahariya": "Northern District",
    "Safed": "Northern District",
    "Kiryat Shmona": "Northern District",
    "Afula": "Northern District",
    "Karmiel": "Northern District",
    "Ma'alot-Tarshiha": "Northern District",
    "Migdal HaEmek": "Northern District",
    "Yokneam Illit": "Northern District",
    "Ashdod": "Southern District",
    "Ashkelon": "Southern District",
    "Beer Sheva": "Southern District",
    "Eilat": "Southern District",
    "Sderot": "Southern District",
    "Ofakim": "Southern District",
    "Dimona": "Southern District",
    "Arad": "Southern District",
    "Kiryat Gat": "Southern District",
    "Kiryat Malakhi": "Southern District"
}

# Simplified mapping based on keywords in the 'region' column
district_mapping_patterns = {
    'Dan': 'Tel Aviv District',
    'Sharon': 'Central District',
    'Hefer': 'Central District',
    'Yarkon': 'Central District',
    'Drom HaSharon': 'Central District',
    'Haifa': 'Haifa District',
    'Pardes Hanna': 'Haifa District',
    'Karkur': 'Haifa District',
    'Hof HaCarmel': 'Haifa District',
    'Menashe': 'Haifa District',
    'Hakrayot': 'Haifa District',
    'Wadi Ara': 'Haifa District',
    'Jerusalem': 'Jerusalem District',
    'Maale Adumim': 'Jerusalem District',
    'Beit Shemesh': 'Jerusalem District',
    'Samaria': 'Judea and Samaria Area',
    'Shomron': 'Judea and Samaria Area',
    'Judea': 'Judea and Samaria Area',
    'Yehuda': 'Judea and Samaria Area',
    'Shfelat Yehuda': 'Judea and Samaria Area',
    'Lakhish': 'Southern District',
    'Gaza': 'Southern District',
    'Confrontation': 'Southern District',
    'Nirim': 'Southern District',
    'Nir Am': 'Southern District',
    'Eilat': 'Southern District',
    'Arava': 'Southern District',
    'Dead Sea': 'Southern District',
    'Negev': 'Southern District',
    'Western Negev': 'Southern District',
    'Safed': 'Northern District',
    'Galilee': 'Northern District',
    'Golan': 'Northern District',
    'Tavor': 'Northern District',
    'HaAmakim': 'Northern District',
    "Beit She'an": 'Northern District',
    'Lev Ha-Hula': 'Northern District',
    'Shfela': 'Central District',
    'Southern Shfela': 'Central District',
    'HaMifratz': 'Haifa District',
    'Ibim': 'Southern District',
    'Gaza Envelope': 'Southern District',
    'West Lachish': 'Southern District',
    'Center Galilee': 'Northern District',
    'Center Negev': 'Southern District',
    'South Golan': 'Northern District',
    'Lower Galilee': 'Northern District',
    'Upper Galilee': 'Northern District',
    'North Golan': 'Northern District', 
    "Beit She'an Valley": 'Northern District',
    'Fast Lane Parking Lot': 'Central District',
    'Mini Israel': 'Central District',
    'Modiin Maccabim Reut': 'Central District',
    'Neot Kedumim': 'Central District',
    'Regem Industrial Zone': 'Southern District',
    'Yehud Monoson': 'Central District',
    'Orevim Cliff': 'Northern District',
    'Ramat Trump': 'Northern District',
    'Jordan River Rafting': 'Northern District',
    'Kfar Naḥum': 'Northern District',
    'Tabgha': 'Northern District',
    "Arabah 310": "Southern District",
    "Ye'arut HaCarmel (Carmel Forest)": "Haifa District",
    "Mivtachim Ami'oz Yesha": "Central District",
    "Kokhav Michael": "Central District",
    "Lachish": "Southern District",
    "Bika'a": "Central District",
    "HaCarmel": "Haifa District",
    "Kfar Yehoshua Train Station": "Central District",
    "Shoham": "Central District",
    "Nili": "Central District",
    "Airport City": "Central District",
    'Goren Guest Farm': 'Northern District',
    'Miluot North Industrial Zone': 'Northern District',
    'Jordan Estate Hotel': 'Northern District',
    'Milouot Industrial Zone North': 'Northern District'
}


In [ ]:
# Function to match keywords in the 'region' column and assign districts
def map_to_district(region):
    for keyword, district in district_mapping_patterns.items():
        if keyword.lower() in region.lower():
            return district
    return 'Undefined'  # Default if no match is found

# Apply logic to create the 'district' column
detailed_df['district'] = detailed_df.apply(
    lambda row: "Southern District" if "ashdod" in row['locality'].lower()  # Ensure all 'Ashdod' locations are categorized correctly
                else city_to_district[row['locality']] if row['locality'] in city_to_district  # Match exact locality if found in city_to_district
                else map_to_district(row['region']),  # Otherwise, try mapping based on region keywords
    axis=1
)

# Drop the 'year' column as it's not needed
detailed_df = detailed_df.drop(columns=['year'])

# Reset the index for better data structure
detailed_df = detailed_df.reset_index(drop=True)

# Display the updated DataFrame
detailed_df


In [ ]:
# Check 'Undefined' district
detailed_df[detailed_df['district'] == 'Undefined']['locality'].unique()

In [ ]:
detailed_df[detailed_df['district'] == 'Undefined']['region'].unique()

In [ ]:
detailed_df.info()

### Saving the dataframe

In [ ]:
# Save the cleaned dataset to a CSV file
detailed_df.to_csv('cumta_detailed_df.csv', index=False, encoding='utf-8')

**Summary of Detailed Dataset Processing and Cleaning**

1️⃣ **Dataset Creation**
- Constructed a dataset with **detailed locality information** based on `raw_df`.

2️⃣ **Yearly Data Distribution Analysis**
- Identified a significant discrepancy in the **distribution of alerts by year** between `detailed_df` and `raw_df`:
  - **In `detailed_df`**: 2021 accounts for **35.09%** of all records.
  - **In `raw_df`**: 2021 makes up only **14.06%** of all records.
- Given that the original dataset consists of **15 files**, and **2021 alert data spans less than 2 files (~13-14%)**, the distribution in `detailed_df` appeared skewed.

3️⃣ **Localization Processing**
- Checked **localities and regions in Hebrew** and translated them into **English**.

4️⃣ **City Name Standardization**
- Identified **large cities** that were divided into **zones**.
- Verified how many such **zones exist** and ensured proper classification.
- Paid special attention to **Ashdod**, where zone names had **non-standard formats**.

5️⃣ **Handling Data Skew and Duplication in 2021**
- Found that **several consecutive lines** contained **the same locality names** (e.g., *Netiv HaAssara, Kissufim*).
- Discovered a **message dated May 20, 2021**, stating:

  > “Fixed a bug that caused duplicate alerts.”

- This confirmed that **prior to May 20, 2021**, **duplicate alert blocks** existed, where **identical localities and timestamps** were repeated **within the same alert**.
- This duplication issue **skewed the yearly distribution**.

6️⃣ **Deduplication & Final Data Cleaning**
- **Removed duplicate alert records** from **before May 20, 2021**.
- Successfully restored the **yearly distribution** to reflect **the original dataset structure**.

---

✅ **Final Result:** The dataset is now **cleaned, properly localized, and free from duplicate alerts** that previously distorted historical distributions.


- **Total rows:** 61281
- **Duplicates:** None  
- **Time column (`'time'`):** Converted to `datetime64[ns]` format  
- **Missing value** None  

**Columns:**

| Column Name | Description |
|-------------|------------|
| **datetime** | Date and time of the warning in `datetime64[ns]` format |
| **locality** | Name of the locality (city, town, or settlement) |
| **region** | Region to which the locality belongs |
| **threat_type** | Type of warning. Possible values: `'Red Alert'`, `'Unrecognized Aircraft'`, `'Terrorist Infiltration'`, `'Interception Pieces'`, `'Earthquake'` |
| **district** | Administrative district or region where the locality is located; an official governmental division. |

## Creating a dataset with an indication of detailed localities (zones of large cities are combined into one record)

In [ ]:
# List of major cities
major_cities_list = [
    "Modi'in", "Hadera", "Caesarea", "Ramat Gan", "Haifa", "Beer Sheva", "Netanya",
    "Rishon LeZion", "Tel Aviv", "Ashdod", "Herzeliya", "Jerusalem", "Nahariya", 
    "Safed", "Dimona", "Kiryat Gat", "Yokneam Illit", "Acre", "Ashkelon", "Kiryat Bialik"
]

# Map for special cases
special_cases = {
    'Hatzor Ashdod': 'Hatzor Ashdod',
    'Hevel Modi\'in': 'Hevel Modi\'in',
    'Modi\'in Illit': 'Modi\'in Illit',
    'Haifa - Kiryat Haim & Kiryat Shmuel': 'Haifa - Kiryat Haim & Kiryat Shmuel'
}

# Function to determine locality and district label
def map_to_locality_and_district(row):
    locality = row['locality']
    # Check if the locality is in special cases
    if locality in special_cases:
        return special_cases[locality], "all"
    # Check if the locality contains a major city name
    for city in major_cities_list:
        if city in locality:
            # Remove city name and clean up
            district = locality.replace(city, "").strip(" -")
            # Remove parentheses using regex
            district = re.sub(r'[()]', '', district).strip()
            return city, district  # Return city and cleaned district name
    # If not in major cities, mark it as its own locality with "all" zones
    return locality, "All"

# Step 1: Apply the mapping logic to determine locality and district
detailed_df[['locality', 'district']] = detailed_df.apply(
    lambda row: pd.Series(map_to_locality_and_district(row)), axis=1
)

# Step 2: Group by 'time' and 'locality', and aggregate zones
df = (
    detailed_df.groupby(['datetime', 'locality'])
    .agg({
        'district': lambda x: ', '.join(
            filter(None, [item.strip() for item in x.unique()])
        ).strip(','),  # Combine unique zones, clean up commas and spaces
        'region': 'first',  # Take the first region as an example
        'threat_type': 'first',  # Take the first threat_type
    })
    .reset_index()
)

# Rename 'district' column for clarity
df.rename(columns={'district': 'zones'}, inplace=True)

# Step 3: Replace empty 'zones' with 'All'
df['zones'] = df['zones'].apply(lambda x: 'All' if not x.strip() else x)

# Step 4: Remove numbers from the 'region' column if locality is in major_cities_list
def clean_region(region, locality):
    if locality in major_cities_list:
        return re.sub(r'\d+', '', region).strip()
    return region

df['region'] = df.apply(lambda row: clean_region(row['region'], row['locality']), axis=1)


In [ ]:
df

### Checking the main cities in the final dataset

In [ ]:
df[df['locality'] == 'Tel Aviv']

In [ ]:
df[df['locality'] == 'Haifa']

In [ ]:
df[df['locality'] == "Modi'in"]

In [ ]:
df[df['locality'] == 'Hadera']

In [ ]:
df[df['locality'] == 'Caesarea']

In [ ]:
df[df['locality'] == 'Ramat Gan']

In [ ]:
df[df['locality'] == 'Beer Sheva']

In [ ]:
df[df['locality'] == 'Rishon LeZion']

In [ ]:
df[df['locality'] == 'Ashdod']

In [ ]:
df[df['locality'] == 'Ashkelon']

In [ ]:
df[df['locality'] == 'Herzeliya']

Сhecking if the dataset contains the words 'Herzliya' instead of 'Herzeliya'

In [ ]:
df[df['locality'] == 'Herzliya']

In [ ]:
# Replace
df.replace('Herzliya', 'Herzeliya', inplace=True)

In [ ]:
# Check
df[df['locality'] == 'Herzliya']

Everything is OK now

In [ ]:
df[df['locality'] == "Ra'anana"]

In [ ]:
df[df['locality'] == 'Jerusalem']

In [ ]:
df[df['locality'] == 'Netanya']

In [ ]:
df[df['locality'] == 'Petach Tikva']

⚠ **WARNING!** ⚠ 
Some localities may refer to different original regions at different times (for example: Herzliya may be attributed to Sharon and Dan, Petah Tikva to Yarkon and Dan). This is due to changes in the data in the telegram channel.


### Checking for duplicates

In [ ]:
print('Number of duplicates:', df.duplicated().sum())

In [ ]:
df.info()

### Saving the dataframe

In [ ]:
df.to_csv('cumta_df.csv', index=False, encoding='utf-8')

## Conclusion
**# Data Preprocessing Proces**s:

- Initially, we constructed a dataframe where each row corresponded to a single message from the Telegram channel. Each message contained information about warnings spanning several minutes.
- Next, we transformed the dataframe so that each row represented a locality. We then merged zones of major cities, assigning the city name to the `'locality'` column, while district names were stored in the `'zones'` column. 
- If a locality was not a major city, the `'zones'` column was set to `'Al**

**Final Processed Dataframe:**

- **Total rows:** 57543 
- **Duplicates:** None  
- **Time column (`'time'`):** Converted to `datetime64[ns]` format  
- **Missing value** None  

**Columns:**

| Column Name | Description |
|-------------|------------|
| **datetime** | Date and time of the warning in `datetime64[ns]` format |
| **locality** | Name of the locality (city, town, or settlement) |
| **zones** | Zones within a locality. If the locality is not a major city, this column contains `'All'` |
| **region** | Region to which the locality belongs |
| **threat_type** | Type of warning. Possible values: `'Red Alert'`, `'Unrecognized Aircraft'`, `'Terrorist Infiltration'`, `'Interception Pieces'`, `'Earthquake'` |
| **district** | Administrative district or region where the locality is located; an official governmental division. |

This structured dataset is now ready for further analysis and visualization.
